In [2]:
import os
import pretty_midi
import numpy as np

In [4]:
# Path to the directory containing MIDI files
base_dir = './Composer_Dataset/NN_midi_files_extended/train'

# List of folders (composers) within the base directory
composer_folders = os.listdir(base_dir)

# Initialize a list to store all loaded MIDI data
all_midi_data = []

# Loop through each composer folder
for composer_folder in composer_folders:
    composer_path = os.path.join(base_dir, composer_folder)
    
    # Check if the item in the base directory is a folder
    if os.path.isdir(composer_path):
        print(f"Loading MIDI files from {composer_folder}...")
        
        # Loop through each MIDI file in the composer folder
        for file_name in os.listdir(composer_path):
            file_path = os.path.join(composer_path, file_name)
            
            # Check if the item in the composer folder is a file and has a .mid or .midi extension
            if os.path.isfile(file_path) and (file_name.endswith('.mid')):
                midi_data = pretty_midi.PrettyMIDI(file_path)
                all_midi_data.append({
                    'composer': composer_folder,
                    'file_name': file_name,
                    'midi_data': midi_data
                })
                print(f"  Loaded: {file_name}")


print("All MIDI files loaded successfully!")

Loading MIDI files from mozart...
  Loaded: mozart048.mid


/opt/anaconda3/envs/tf2/lib/python3.8/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


  Loaded: mozart028.mid
  Loaded: mozart000.mid
  Loaded: mozart001.mid
  Loaded: mozart015.mid
  Loaded: mozart029.mid
  Loaded: mozart003.mid
  Loaded: mozart017.mid
  Loaded: mozart016.mid
  Loaded: mozart002.mid
  Loaded: mozart006.mid
  Loaded: mozart012.mid
  Loaded: mozart013.mid
  Loaded: mozart007.mid
  Loaded: mozart011.mid
  Loaded: mozart005.mid
  Loaded: mozart010.mid
  Loaded: mozart009.mid
  Loaded: mozart021.mid
  Loaded: mozart034.mid
  Loaded: mozart008.mid
  Loaded: mozart022.mid
  Loaded: mozart036.mid
  Loaded: mozart037.mid
  Loaded: mozart023.mid
  Loaded: mozart027.mid
  Loaded: mozart033.mid
  Loaded: mozart032.mid
  Loaded: mozart026.mid
  Loaded: mozart030.mid
  Loaded: mozart024.mid
  Loaded: mozart018.mid
  Loaded: mozart019.mid
  Loaded: mozart031.mid
  Loaded: mozart042.mid
  Loaded: mozart043.mid
  Loaded: mozart041.mid
  Loaded: mozart044.mid
  Loaded: mozart045.mid
  Loaded: mozart047.mid
  Loaded: mozart046.mid
Loading MIDI files from chopin...
  Load

KeyboardInterrupt: 

In [15]:
# Example of extracing midi file metadata
composer = all_midi_data[0]['composer']
print(f"Composer (from all_midi_data[0]): {composer}")


Composer (from all_midi_data[0]): mozart


In [8]:
# Path to the directory containing MIDI files
base_dir = './Composer_Dataset/NN_midi_files_extended/train'

# List of folders (composers) within the base directory
composer_folders = os.listdir(base_dir)

# Initialize a list to store all loaded MIDI data
all_midi_data = []

# Loop through each composer folder
for composer_folder in composer_folders:
    composer_path = os.path.join(base_dir, composer_folder)
    
    # Check if the item in the base directory is a folder
    if os.path.isdir(composer_path):
        print(f"Loading MIDI files from {composer_folder}...")
        
        # Loop through each MIDI file in the composer folder
        for file_name in os.listdir(composer_path):
            file_path = os.path.join(composer_path, file_name)
            
            # Check if the item in the composer folder is a file and has a .mid or .midi extension
            if os.path.isfile(file_path) and (file_name.endswith('.mid')):
                midi_data = pretty_midi.PrettyMIDI(file_path)
                all_midi_data.append({
                    'composer': composer_folder,
                    'file_name': file_name,
                    'midi_data': midi_data
                })
                print(f"Loaded: {file_name}")

print("All MIDI files loaded successfully!")

Loading MIDI files from mozart...
Loaded: mozart048.mid
Loaded: mozart028.mid
Loaded: mozart000.mid
Loaded: mozart001.mid
Loaded: mozart015.mid
Loaded: mozart029.mid
Loaded: mozart003.mid
Loaded: mozart017.mid
Loaded: mozart016.mid
Loaded: mozart002.mid
Loaded: mozart006.mid
Loaded: mozart012.mid
Loaded: mozart013.mid
Loaded: mozart007.mid
Loaded: mozart011.mid
Loaded: mozart005.mid
Loaded: mozart010.mid
Loaded: mozart009.mid
Loaded: mozart021.mid
Loaded: mozart034.mid
Loaded: mozart008.mid
Loaded: mozart022.mid
Loaded: mozart036.mid
Loaded: mozart037.mid
Loaded: mozart023.mid
Loaded: mozart027.mid
Loaded: mozart033.mid
Loaded: mozart032.mid
Loaded: mozart026.mid
Loaded: mozart030.mid
Loaded: mozart024.mid
Loaded: mozart018.mid
Loaded: mozart019.mid
Loaded: mozart031.mid
Loaded: mozart042.mid
Loaded: mozart043.mid
Loaded: mozart041.mid
Loaded: mozart044.mid
Loaded: mozart045.mid
Loaded: mozart047.mid
Loaded: mozart046.mid
Loading MIDI files from chopin...
Loaded: chopin054.mid
Loaded: 